In [73]:
import numpy as np
import pandas as pd
import os

In [74]:
testdf=pd.read_csv('coursera_sessions_test.txt',sep=';',header=None,names=['v','b'],dtype={'v': str, 'b': str})
traindf=pd.read_csv('coursera_sessions_train.txt',sep=';',header=None,names=['v','b'],dtype={'v': str, 'b': str})
traindf.head(10)

,v,b
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN
5,42,NaN
6,"47,48,49",NaN
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
8,"71,72,73,74",NaN
9,"76,77,78",NaN


Расчитаем частоты появляения id в обучающей выборке

In [75]:
# Функция считает количество появлений каждого ИД в наборе.
# Под набором (sessions) понимаем список просмотренных или купленных товарах во всей выборке
def get_num(sessions):
    # Формируем список всех встречающихся ИД в наборе
    l=list()
    for session in sessions:
        l += session.split(',')
    l=list(map(int,l))
    # Считаем сколько раз встретился каждый ИД в наборе
    freq = np.bincount(l)
    ii = np.nonzero(freq)[0]
    # Возвращаем список - (ИД - количество раз сколько он встретился в списке)
    return list(zip(ii,freq[ii]))

In [77]:
# Формируем для каждого ИД количество его наблюдений в просмотре и покупке на обучении
# Считаем количество в просмотрах
rate_df=pd.DataFrame(get_num(list(traindf.v)),columns=['id','view_num'])
rate_df=rate_df.set_index('id')
# Считаем количество в покупках
rate_df1=pd.DataFrame(get_num(list(traindf.b.dropna())),columns=['id','buy_num'])
rate_df1=rate_df1.set_index('id')
# Объединяем, считаем, что если товара не было в покупках, то он не покупался
rate_df=rate_df.join(rate_df1)
rate_df=rate_df.fillna(0)

In [78]:
rate_df

,view_num,buy_num
id,,
0,6,0.0
1,6,0.0
2,9,0.0
3,7,0.0
4,11,0.0
...,...,...
102802,2,0.0
102803,1,0.0
102804,1,0.0


In [80]:
# Добавляем  ИД, которые есть в тесте, но нет в обучении
# Получаем список ИД в тесте
test_ids=list()
for s in list(testdf.v):
    test_ids+=s.split(',')
test_ids=list(map(int,test_ids))
# Определяем те, которых нет в обучении
dif=set(test_ids).difference(list(rate_df.index))
dif=list(dif)
dfdif=pd.DataFrame(index=dif,columns=['view_num','buy_num'])
dfdif.view_num=0
dfdif.buy_num=0
rate_df=rate_df.append(dfdif)


In [81]:
# Считаем для каждого ИД его ранк относительно просмотренных и купленных товаров
rate_df['view_rank']=rate_df.view_num.rank(ascending=False)
rate_df['buy_rank']=rate_df.buy_num.rank(ascending=False)

In [82]:
rate_df.head()

,view_num,buy_num,view_rank,buy_rank
0,6,0.0,14211.5,53509.5
1,6,0.0,14211.5,53509.5
2,9,0.0,8828.0,53509.5
3,7,0.0,11933.5,53509.5
4,11,0.0,6734.0,53509.5


Реализуйте два алгоритма рекомендаций:

* сортировка просмотренных id по популярности (частота появления в просмотренных),
* сортировка просмотренных id по покупаемости (частота появления в покупках).

In [84]:
# Функция получает на вход ИД view_ids и возвращает k из них, в соотвествии с алгоритмом рекомендации alg
# view_ids - просмотренные товары
# k - количество рекомендаций для возврата
# alg - алгорим построения рекомендаций (view_rank - по популярности, buy_rank - по покупаемости)
def get_recom_id(view_ids,k,alg):
    # список уникальных ИД
    view=set(view_ids)
    # минимум из К и количеством товаров
    first=min(k,len(view_ids))
    # возвращаем топ лучших ИД
    return list(rate_df.loc[view,alg].sort_values(kind='mergesort')[0:first].index)

In [85]:
# Функция расчитывает для сессии [view,buy] метрики precision(k), recall(k) исходя из алгоритма рекомендаций alg
def metric(view,buy,k,alg):
    view_ids=view.split(',')
    view_ids=list(map(int,view_ids))
    buy_ids=buy.split(',')
    buy_ids=list(map(int,buy_ids))
    # получаем список рекомендаций для сессии
    recom=get_recom_id(view_ids,k,alg)
    # Считаем количество покупок из рекомендованных
    purchases=0
    for id_ in buy_ids:
        if id_ in recom:
            purchases+=1
    #print ("buy=%s, recom=%s, purchases=%s"%(buy_ids,recom,purchases))
    # Precision = Количество покупок из рекомендованных / k
    precision=purchases/k
    # Recall = Количество покупок из рекомендованных / Количество покупок
    recall=purchases/len(buy_ids)
    return precision,recall

In [86]:
# Функция считает метрики precision(k), recall(k) для сессий в DF
def calc_metrics(df):
    for session in df[df.b.notnull()].iterrows():
        id_=session[0]
        view=session[1]['v']
        buy=session[1]['b']
        for k in [1,5]:
            for alg in ['view_rank','buy_rank']:
                p,r=metric(view,buy,k,alg)
                df.loc[id_,'p'+str(k)+alg]=p
                df.loc[id_,'r'+str(k)+alg]=r

In [87]:
calc_metrics(traindf)
calc_metrics(testdf)

In [88]:
def get_avg_metrics(df):
    res=pd.DataFrame(columns=['view_rank','buy_rank'])
    for k in [1,5]:
        for metric in ['r','p']:
            for alg in ['view_rank','buy_rank']:
                res.loc[metric+str(k),alg]=round(df[metric+str(k)+alg].mean(skipna=True),2)
    return res

In [89]:
t1=get_avg_metrics(traindf)
t1

,view_rank,buy_rank
r1,0.44,0.68
p1,0.51,0.79
r5,0.82,0.93
p5,0.21,0.25


In [91]:
t2=get_avg_metrics(testdf)
t2


,view_rank,buy_rank
r1,0.42,0.41
p1,0.48,0.47
r5,0.8,0.79
p5,0.2,0.2
